In [1]:
# LT 21/09/2022
# extract mutational model parameters from RMC project

import hail as hl
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'regional_missense_constraint',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

2022-09-21 08:00:16 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://6d05993bf632:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.98-f8833c1ae16b
LOGGING: writing to /home/jupyter/structural-constraint/gnomad/prototype/hail-20220921-0800-0.2.98-f8833c1ae16b.log


In [8]:
# RMC results table
ht = hl.read_table('gs://regional_missense_constraint/model/2.1.1/1/context_obs_exp_annot.ht')

In [9]:
# filter to ACTA1 canonical transcript
ht = ht.filter(ht.transcript=="ENST00000366684").persist()

In [12]:
# annotate with the cumulative exp of previous row
ht = ht.annotate(prev_row_cumexp=hl.scan.take(ht.cumulative_exp,1,ordering=-ht.cumulative_exp).first())

# calculate locus-specific expected by taking the difference
ht = ht.annotate(exp = ht.cumulative_exp - hl.coalesce(ht.prev_row_cumexp,0))

ht.show()
# keep what we need and rename observed to obs

#ht = ht.transmute(obs = ht.observed)



,,,,,,,,,,,,,,,
locus,transcript,mu_snp,coverage,total_exp,total_mu,total_obs,overall_oe,cumulative_obs,cumulative_exp,forward_oe,obs_scan,mu_scan,prev_row_cumexp,exp,obs
locus<GRCh37>,str,float64,int32,float64,float64,int64,float64,int64,float64,float64,int64,float64,float64,float64,int32
1:229567249,"""ENST00000366684""",6.25e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,9.25e-02,0.00e+00,NA,6.25e-09,NA,9.25e-02,0
1:229567250,"""ENST00000366684""",5.17e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,1.69e-01,0.00e+00,0,1.14e-08,9.25e-02,7.66e-02,0
1:229567251,"""ENST00000366684""",6.37e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,2.63e-01,0.00e+00,0,1.78e-08,1.69e-01,9.43e-02,0
1:229567252,"""ENST00000366684""",2.30e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,2.98e-01,0.00e+00,0,2.01e-08,2.63e-01,3.41e-02,0
1:229567253,"""ENST00000366684""",1.19e-08,100,2.61e+02,1.76e-05,55,2.11e-01,0,4.74e-01,0.00e+00,0,3.20e-08,2.98e-01,1.76e-01,0
1:229567254,"""ENST00000366684""",1.56e-08,100,2.61e+02,1.76e-05,55,2.11e-01,0,7.05e-01,0.00e+00,0,4.76e-08,4.74e-01,2.31e-01,0
1:229567255,"""ENST00000366684""",2.78e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,7.46e-01,0.00e+00,0,5.04e-08,7.05e-01,4.12e-02,0
1:229567256,"""ENST00000366684""",5.93e-09,100,2.61e+02,1.76e-05,55,2.11e-01,0,8.34e-01,0.00e+00,0,5.63e-08,7.46e-01,8.77e-02,0


In [13]:
ht = ht.select(ht.obs, ht.exp)
# export
ht.export("ENST00000366684_bp_oe.tsv")

2022-09-21 08:52:04 Hail: INFO: merging 16402 files totalling 37.2K...) / 16401]
2022-09-21 08:52:06 Hail: INFO: while writing:
    ENST00000366684_bp_oe.tsv
  merge time: 2.481s
